In [1]:
import pandas as pd
# from tabulate import tabulate
import numpy as np
from pygobnilp.gobnilp import Gobnilp
# import bnlearn as bn
import pgmpy
import itertools
from scipy.stats import chi2_contingency #check for independence between 2 variables
from scipy.stats import power_divergence
from pgmpy.estimators.CITests import log_likelihood
from pgmpy.estimators.CITests import chi_square
import collections
# from pgmpy.factors.continuous.discretize import BaseDiscretizer

In [2]:
def csv_format_discrete(csv_file):
    df = pd.read_csv(csv_file, sep="\s+")
    return df.drop([0])

#returns the csv_file in a pandas dataframe, formatted properly, discrete dataset only

In [15]:
df = csv_format_discrete(r"C:\Users\User\Documents\GitHub\ML_FYP\dataset\alarm_100.dat")
df
df_test = csv_format_discrete(r"C:\Users\User\Documents\GitHub\ML_FYP\dataset\alarm_10000.dat")
df_test

,PULMEMBOLUS,PAP,KINKEDTUBE,INTUBATION,MINVOLSET,VENTMACH,DISCONNECT,VENTTUBE,VENTLUNG,SHUNT,...,CO,BP,EXPCO2,ERRLOWOUTPUT,PRESS,HRBP,MINVOL,HISTORY,HRSAT,PCWP
1,1,1,0,1,2,2,1,3,3,0,...,1,1,1,0,0,0,3,0,0,2
2,0,2,0,1,2,2,0,1,3,1,...,0,0,1,1,1,2,3,0,0,2
3,0,2,0,2,2,2,0,1,1,0,...,0,0,2,0,0,0,1,0,0,2
4,0,2,0,1,2,2,0,1,3,1,...,1,1,1,0,0,0,3,1,2,1
5,0,2,0,1,2,2,0,1,3,1,...,0,0,1,0,1,1,3,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,0,2,1,1,2,2,0,1,3,1,...,1,1,1,0,1,0,3,0,0,0
9997,0,0,0,1,2,2,0,1,3,1,...,0,0,1,0,2,0,3,0,0,2
9998,0,2,0,0,0,0,0,0,2,1,...,0,0,2,0,0,0,3,0,0,2
9999,0,2,0,1,2,2,0,1,3,1,...,1,1,1,0,1,0,3,0,0,0


### chi-squared test

In [48]:
#0th order chi2 test
def chi2bool(df):
    '''
    0th order CI test
    '''
    chi2 = []
    v = list(df)
    my_list = list(itertools.combinations(v,2))
    y_0, y_1 = [x[0] for x in my_list], [x[1] for x in my_list]
    for i,j in zip(y_0, y_1):
        chi = chi_square(X=i, Y=j, Z=[], data=df, significance_level=0.05)
        chi2.append((i,j,chi))
    true_0 = [x for x in chi2 if True in x]
    false_0 = [x for x in chi2 if False in x]
    return true_0, false_0

### Sorting Functions

In [47]:
def create_permutations(my_list):
    '''
    takes in a list, remove extra permutations from the list and only creates a pair of permutations
    '''
    p_permutations = []
    new_list = list(set(tuple(sorted(l[:2])) for l in my_list))
    for items in new_list:
        new_list_0 = list(itertools.permutations(items[:2],2))[1]
        p_permutations.append(new_list_0)
    return new_list + p_permutations

In [49]:
#given 2 lists, append them together and remove duplicates
def remove_dupes(list_1, list_2):
    '''
    unsure
    '''
    for i in list_2:
        list_1.append(i)
    remove_dupes = list(set(list_1))
    return remove_dupes

### CONDITIONAL MAIN CODE

In [80]:
#1st order CI
def cond_1_generate(df):
    v = list(df)
    p_permutations = list(itertools.permutations(v,3))
    order_0 = create_permutations(chi2bool(df)[0])
    generated = [x for x in p_permutations if x[:2] not in order_0]
    generated_0 = set(tuple(sorted(items[:2])) for items in generated)
    generate_return = [x for x in generated if x[:2] in generated_0]
    return generate_return

In [81]:
def cond_1_test(df):
    chi2_data = []
    phi = cond_1_generate(df)
    for i,j,k in phi:
        chi2 = chi_square(X=i, Y=j, Z=[k], data=df, significance_level=0.05)
        chi2_data.append((i,j,k,chi2))
    true_list = [x for x in chi2_data if True in x]
    false_list = [x for x in chi2_data if False in x]
    return true_list, false_list

In [82]:
%%time
G = create_permutations(chi2bool(df)[0])
len(G), len(chi2bool(df)[0])

CPU times: total: 1.38 s
Wall time: 1.4 s


(1040, 520)

In [84]:
chi_0, chi_1  = [x[:2] for x in chi2bool(df)[0]],  [x[:2] for x in cond_1_test(df)[0]]
chi_2 = chi_0 + chi_1
len(chi_0), len(chi_1), len(chi_2)

(520, 773, 1293)

In [95]:
len(create_permutations(chi_1))

230

In [87]:
p_permute = create_permutations(chi_2)

In [97]:
len(p_permute)

1270

In [93]:
%%time
N = 3
n = 3
v = list(df)
# remove_list = [x for x in p_permute]
remove_list = create_permutations([x[:2] for x in chi2bool(df)[0]])
empty_list = []
while N <= n:
    list_permutations = [x for x in itertools.permutations(v, N) if x[:2] not in remove_list]
    p_1 = [x[:1] for x in list_permutations]
    p_2 = [x[1:2] for x in list_permutations]
    p = [x[2:] for x in list_permutations]
    for i,j,k in zip(p_1, p_2, p):
        chi2 = chi_square(X=i[0], Y=j[0], Z=k, data=df, significance_level=0.95)
        empty_list.append((i[0],j[0],k,chi2))
    true_list = create_permutations([x[:2] for x in empty_list if True in x])
    remove_list = remove_list + create_permutations(true_list)
    remove_list = create_permutations(remove_list)
    N += 1

CPU times: total: 38.3 s
Wall time: 38.7 s


In [104]:
p0 = [x for x in p_permute if x not in remove_list]
(p0)

[('TwentyFive', 'TwentyNine'),
 ('TwentyNine', 'TwentySeven'),
 ('ThirtyFive', 'TwentyNine'),
 ('TwentyEight', 'TwentyFour'),
 ('ThirtySeven', 'TwentySeven'),
 ('Eleven', 'Fourteen'),
 ('Twenty', 'TwentyEight'),
 ('ThirtySeven', 'TwentyFour'),
 ('Fourteen', 'TwentyTwo'),
 ('TwentyFour', 'TwentySeven'),
 ('TwentyNine', 'TwentyThree'),
 ('ThirtyFive', 'TwentyFive'),
 ('Seventeen', 'TwentySix'),
 ('Five', 'ThirtyFour'),
 ('Eight', 'Fourteen'),
 ('Six', 'ThirtyFour'),
 ('ThirtySix', 'ThirtyThree'),
 ('Eleven', 'Nineteen'),
 ('Eleven', 'Six'),
 ('Nineteen', 'TwentyEight'),
 ('Seven', 'ThirtyFour'),
 ('Eight', 'ThirtyFour'),
 ('ThirtyOne', 'Two'),
 ('TwentySeven', 'TwentyThree'),
 ('Five', 'Sixteen'),
 ('TwentyEight', 'TwentyTwo'),
 ('Six', 'Sixteen'),
 ('ThirtyThree', 'TwentySix'),
 ('TwentyFive', 'TwentyFour'),
 ('Five', 'Thirteen'),
 ('Thirteen', 'TwentyTwo'),
 ('ThirtyThree', 'TwentyEight'),
 ('Eighteen', 'Nineteen'),
 ('Eighteen', 'ThirtyThree'),
 ('ThirtySix', 'TwentyEight'),
 ('Ninete

In [94]:
len(remove_list)

1116

In [68]:
remove_0 = [create_permutations(x[:2] for x in chi2bool(df)[0])]
remove_0

[[('Three', 'TwentyThree'),
  ('One', 'Three'),
  ('Fifteen', 'ThirtyFour'),
  ('ThirtyTwo', 'Three'),
  ('Eleven', 'Twelve'),
  ('Eighteen', 'ThirtyFive'),
  ('One', 'Ten'),
  ('ThirtyOne', 'TwentyThree'),
  ('One', 'TwentyOne'),
  ('ThirtyTwo', 'TwentyOne'),
  ('ThirtySix', 'TwentyFive'),
  ('Seventeen', 'TwentyNine'),
  ('Five', 'Two'),
  ('Sixteen', 'TwentyEight'),
  ('Sixteen', 'ThirtySeven'),
  ('Eleven', 'TwentyFive'),
  ('Eighteen', 'Four'),
  ('Five', 'ThirtySix'),
  ('Three', 'Twelve'),
  ('Ten', 'TwentySix'),
  ('Nineteen', 'ThirtyFive'),
  ('Six', 'Two'),
  ('Thirteen', 'ThirtyOne'),
  ('Twelve', 'TwentySeven'),
  ('Twenty', 'TwentyFive'),
  ('Seventeen', 'Three'),
  ('ThirtyOne', 'Twelve'),
  ('Six', 'ThirtySix'),
  ('Fourteen', 'Twenty'),
  ('Nine', 'TwentyEight'),
  ('Seventeen', 'Ten'),
  ('Eighteen', 'Three'),
  ('ThirtyFour', 'TwentyTwo'),
  ('Three', 'TwentyFive'),
  ('Seventeen', 'TwentyOne'),
  ('Five', 'Thirty'),
  ('Eighteen', 'Ten'),
  ('TwentyFive', 'TwentyOne'

In [55]:
len(remove_list)

156

In [75]:
list_permutations

[('One', 'Two', 'Three'),
 ('One', 'Two', 'Four'),
 ('One', 'Two', 'Five'),
 ('One', 'Two', 'Six'),
 ('One', 'Two', 'Seven'),
 ('One', 'Two', 'Eight'),
 ('One', 'Two', 'Nine'),
 ('One', 'Two', 'Ten'),
 ('One', 'Two', 'Eleven'),
 ('One', 'Two', 'Twelve'),
 ('One', 'Two', 'Thirteen'),
 ('One', 'Two', 'Fourteen'),
 ('One', 'Two', 'Fifteen'),
 ('One', 'Two', 'Sixteen'),
 ('One', 'Two', 'Seventeen'),
 ('One', 'Two', 'Eighteen'),
 ('One', 'Two', 'Nineteen'),
 ('One', 'Two', 'Twenty'),
 ('One', 'Two', 'TwentyOne'),
 ('One', 'Two', 'TwentyTwo'),
 ('One', 'Two', 'TwentyThree'),
 ('One', 'Two', 'TwentyFour'),
 ('One', 'Two', 'TwentyFive'),
 ('One', 'Two', 'TwentySix'),
 ('One', 'Two', 'TwentySeven'),
 ('One', 'Two', 'TwentyEight'),
 ('One', 'Two', 'TwentyNine'),
 ('One', 'Two', 'Thirty'),
 ('One', 'Two', 'ThirtyOne'),
 ('One', 'Two', 'ThirtyTwo'),
 ('One', 'Two', 'ThirtyThree'),
 ('One', 'Two', 'ThirtyFour'),
 ('One', 'Two', 'ThirtyFive'),
 ('One', 'Two', 'ThirtySix'),
 ('One', 'Two', 'ThirtySev

In [64]:
u0 = [x[:2] for x in empty_list if True in x]
u1 = create_permutations(u0)
u1

[('Eight', 'Nine'),
 ('Twenty', 'TwentyTwo'),
 ('Nine', 'Thirty'),
 ('Eleven', 'ThirtyFour'),
 ('Thirteen', 'ThirtyFour'),
 ('Eleven', 'Thirty'),
 ('Eleven', 'Nine'),
 ('ThirtySix', 'TwentyTwo'),
 ('Nine', 'Eight'),
 ('TwentyTwo', 'Twenty'),
 ('Thirty', 'Nine'),
 ('ThirtyFour', 'Eleven'),
 ('ThirtyFour', 'Thirteen'),
 ('Thirty', 'Eleven'),
 ('Nine', 'Eleven'),
 ('TwentyTwo', 'ThirtySix')]

In [36]:
%%time
empty_list = []
N = 3
list_permutations = [x for x in itertools.permutations(v, N) if x[:2] not in remove_list]
p_1 = [x[:1] for x in list_permutations]
p_2 = [x[1:2] for x in list_permutations]
p = [x[2:] for x in list_permutations]
# true_list = [x for x in empty_list if True in x]
# false_list = [x for x in empty_list if False in x]

CPU times: total: 31.2 ms
Wall time: 38.3 ms


In [52]:
chi2bool(df),conditional_1st_order(df)

([[0, ('One', 'Two'), False],
  [1, ('One', 'Three'), True],
  [2, ('One', 'Four'), False],
  [3, ('One', 'Five'), False],
  [4, ('One', 'Six'), False],
  [5, ('One', 'Seven'), True],
  [6, ('One', 'Eight'), True],
  [7, ('One', 'Nine'), True],
  [8, ('One', 'Ten'), True],
  [9, ('One', 'Eleven'), False],
  [10, ('One', 'Twelve'), True],
  [11, ('One', 'Thirteen'), True],
  [12, ('One', 'Fourteen'), True],
  [13, ('One', 'Fifteen'), True],
  [14, ('One', 'Sixteen'), False],
  [15, ('One', 'Seventeen'), True],
  [16, ('One', 'Eighteen'), True],
  [17, ('One', 'Nineteen'), True],
  [18, ('One', 'Twenty'), True],
  [19, ('One', 'TwentyOne'), True],
  [20, ('One', 'TwentyTwo'), True],
  [21, ('One', 'TwentyThree'), True],
  [22, ('One', 'TwentyFour'), True],
  [23, ('One', 'TwentyFive'), True],
  [24, ('One', 'TwentySix'), True],
  [25, ('One', 'TwentySeven'), True],
  [26, ('One', 'TwentyEight'), True],
  [27, ('One', 'TwentyNine'), True],
  [28, ('One', 'Thirty'), True],
  [29, ('One', '

In [38]:
for j in p_2:
    print(j[0])

Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seve

In [50]:
%%time
tol = []
for i,j,k in zip(p_1, p_2, p):
    chi2 = chi_square(X=i[0], Y=j[0], Z=k, data=df, significance_level=0.05)
    tol.append((i[0],j[0],[x for x in k], chi2))
tol

CPU times: total: 2min 40s
Wall time: 2min 41s


[('One', 'Two', ['Three'], False),
 ('One', 'Two', ['Four'], True),
 ('One', 'Two', ['Five'], True),
 ('One', 'Two', ['Six'], True),
 ('One', 'Two', ['Seven'], False),
 ('One', 'Two', ['Eight'], False),
 ('One', 'Two', ['Nine'], False),
 ('One', 'Two', ['Ten'], True),
 ('One', 'Two', ['Eleven'], True),
 ('One', 'Two', ['Twelve'], False),
 ('One', 'Two', ['Thirteen'], False),
 ('One', 'Two', ['Fourteen'], False),
 ('One', 'Two', ['Fifteen'], False),
 ('One', 'Two', ['Sixteen'], True),
 ('One', 'Two', ['Seventeen'], False),
 ('One', 'Two', ['Eighteen'], False),
 ('One', 'Two', ['Nineteen'], False),
 ('One', 'Two', ['Twenty'], False),
 ('One', 'Two', ['TwentyOne'], False),
 ('One', 'Two', ['TwentyTwo'], False),
 ('One', 'Two', ['TwentyThree'], False),
 ('One', 'Two', ['TwentyFour'], False),
 ('One', 'Two', ['TwentyFive'], False),
 ('One', 'Two', ['TwentySix'], False),
 ('One', 'Two', ['TwentySeven'], False),
 ('One', 'Two', ['TwentyEight'], False),
 ('One', 'Two', ['TwentyNine'], False),


In [97]:
for i,j,k in zip(p_1, p_2, p):
    chi_square(X=i, Y=j, Z=[], data=df, significance_level=0.95)

KeyError: ('One',)

In [84]:
list_permutations

[('One', 'Two', 'Three', 'Four'),
 ('One', 'Two', 'Three', 'Five'),
 ('One', 'Two', 'Three', 'Six'),
 ('One', 'Two', 'Three', 'Seven'),
 ('One', 'Two', 'Three', 'Eight'),
 ('One', 'Two', 'Three', 'Nine'),
 ('One', 'Two', 'Three', 'Ten'),
 ('One', 'Two', 'Three', 'Eleven'),
 ('One', 'Two', 'Three', 'Twelve'),
 ('One', 'Two', 'Three', 'Thirteen'),
 ('One', 'Two', 'Three', 'Fourteen'),
 ('One', 'Two', 'Three', 'Fifteen'),
 ('One', 'Two', 'Three', 'Sixteen'),
 ('One', 'Two', 'Three', 'Seventeen'),
 ('One', 'Two', 'Three', 'Eighteen'),
 ('One', 'Two', 'Three', 'Nineteen'),
 ('One', 'Two', 'Three', 'Twenty'),
 ('One', 'Two', 'Three', 'TwentyOne'),
 ('One', 'Two', 'Three', 'TwentyTwo'),
 ('One', 'Two', 'Three', 'TwentyThree'),
 ('One', 'Two', 'Three', 'TwentyFour'),
 ('One', 'Two', 'Three', 'TwentyFive'),
 ('One', 'Two', 'Three', 'TwentySix'),
 ('One', 'Two', 'Three', 'TwentySeven'),
 ('One', 'Two', 'Three', 'TwentyEight'),
 ('One', 'Two', 'Three', 'TwentyNine'),
 ('One', 'Two', 'Three', 'Thi

In [80]:
%%time
p = [x for x in list_permutations if x[:2] not in remove_list]
len(p)

CPU times: total: 9.44 s
Wall time: 9.47 s


347480